In [216]:
import numpy as np
import pandas as pd
import random
import math
import csv

In [217]:
# import csv of lightpole raw

lightpole_raw = pd.read_csv('lightpole.csv', sep=',', header=None)

# convert to np array

lightpole = np.array(lightpole_raw)

In [218]:
print('x-range: ', min(lightpole[:,0]), max(lightpole[:,0]))
print('y-range: ', min(lightpole[:,1]), max(lightpole[:,1]))
print('z-range: ', min(lightpole[:,2]), max(lightpole[:,2]))

x-range:  976005.21 976007.65
y-range:  172934.38 172940.7
z-range:  45.6 68.81


In [219]:
variances = []

for i in range(3):
    values = [point[i] for point in lightpole]
    var = np.var(values)
    variances.append(var)
    
print(variances)

[0.31199848279188774, 4.328639461263329, 55.10259703797404]


### Octree Implementation from CSV

In [248]:
# used to create an edge.csv for gephi to visualize network
# format: Target, Source, Type:Directed
edge = []

# used to create a node.csv for gephi to understand depth
# format: Id, Label:Id, Depth
node = []

In [249]:
# this is a function for creating unique id for each node

node_ids = set()

def generate_node_id(node_ids):
    new_id = random.randint(1, 1e5)
    while new_id in node_ids:
        new_id = random.randint(1, 1e5)
    node_ids.add(new_id)
    return new_id

In [250]:
# this is a conventional octree careless about the efficiency of data retrieval
# i.e., there is no implementation of the algorithm that determines the most optimal way to create octants

def visualize_octree(data,source_id, depth):
    num_point = len(data)
    
    # create a new id for each node
    global node_ids
    target_id = generate_node_id(node_ids)
    
    # create one row representing a node in the octree network
    global edge
    global node
    edge.append([source_id, target_id, 'Directed'])
    node.append([target_id, target_id, depth])
    
    
    if num_point <= 1:
        # Base case
        return # would fill up the edge array
    elif num_point <= 8:
        # If there are 8 or fewer points, each would occupy an octant
        for i in range(num_point):
            visualize_octree(data[i:i+1], target_id, depth + 1)
    else:
        # Divide the data into 8 octants
        num_point_per_octant = math.floor(num_point / 8)
        count = 0
        for i in range(0, num_point, num_point_per_octant):
            count = count + 1 
            if count >= 8:
                visualize_octree(data[i : num_point], target_id, depth + 1)
                break # break the for loop
            else:
                visualize_octree(data[i : i + num_point_per_octant], target_id, depth + 1)

In [251]:
visualize_octree(lightpole, float('nan'), 0)

In [252]:
pd.DataFrame(edge)

,0,1,2
0,NaN,71939,Directed
1,71939.0,91363,Directed
2,91363.0,78975,Directed
3,78975.0,65189,Directed
4,78975.0,33238,Directed
...,...,...,...
599,30804.0,68773,Directed
600,30804.0,904,Directed
601,30804.0,58193,Directed
602,30804.0,38190,Directed


In [253]:
pd.DataFrame(node)

,0,1,2
0,71939,71939,0
1,91363,91363,1
2,78975,78975,2
3,65189,65189,3
4,33238,33238,3
...,...,...,...
599,68773,68773,4
600,904,904,4
601,58193,58193,4
602,38190,38190,4


In [254]:
edge_header = ['Source', 'Target', 'Type']

with open('edge.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(edge_header)
    writer.writerows(edge[1:])

node_header = ['Id', 'Label', 'Depth']
with open('node.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(node_header)
    writer.writerows(node)

### R-Tree Visualization Implementation

1. Need to constantly adjust the bounding box size for the smaller bounding boxes
2. Another Base Case: the bounding box size = [1,1,1]
3. For now, it is just a rough base case with the whole plot being the root bounding box ... will shrink the root bounding box to a little bit larger than the max of its children bounding box
4. Better be binary tree, so the performance is better
5. Make it more general (already general??)
6. Performance measure

1. Rename the colormap label

1. I need to fix the octree a little bit...

### Octree, K-D Tree, R-Tree Implementation

1. performance measure

### Tool selection (later)
1. What is the software for inputing point cloud data into Octree & K-D tree & R-Tree? 
        * Implement the trees
        * Measure the performance (memory used, speed)
2. What is the tool for measuring the speed of spatial data retrieval?
3. What is the tool for measuring the speed of building an actual Octree & K-D Tree & R-Tree (Do we need to implement the tree structures)